In [34]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


To run the model:

- Click 'Cell' (above), then 'Run All' from the drop down menu.
- A text box will appear below labeled "Scenario to run:"
    - Type 'All' to run all 4 scenarios "baseline, low, medium, and high"
        - Runtime ~ 2 minutes 10 seconds for all 4 scenarios
    - To run only one scenario, type either 'Baseline', 'A' (Low scenario),'B' (Medium Scenario), or 'C' (High Scenario)
        - Runtime ~ 37 seconds to run 1 scenario

In [35]:
import os
import pickle
from pathlib import Path
import pandas as pd
import numpy as np
import xlwings as xw
import copy
import matplotlib.pyplot as plt
%matplotlib inline
plt.ioff()
from matplotlib.ticker import StrMethodFormatter
import time
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import Image
import matplotlib.image as mpimg
from matplotlib import rcParams

plt.rcParams.update({'figure.max_open_warning': 0})

# figure size in inches optional
rcParams['figure.figsize'] = 5.12, 3.84

CSS = """

.output {
  flex-direction: row;
  flex-wrap: wrap;
}

.dropdown-menu{
  max-height:200px;
  overflow:hidden;
  overflow-y:auto;
  width:auto
}
"""

HTML('<style>{}</style>'.format(CSS))



In [36]:
def scenarioanalysis(scenarioletter):
    calyr = range(2019,2036)
    modyr = [i for i in range(1961,2036)]
    calyr = [i for i in range(2019,2036)]
    methane_GWP = 28
    n20_GWP = 265
    dge_to_MJ = 134.47 #MJ/DGE
    gge_to_MJ = 115.83 #MJ/GGE
    gge_to_dge = 0.88 #DGE/GGE
    kWh_to_MJ = 3.6 #MJ/kWh
    CO2target = (14.522  * .2) #MMT CO2e
    
    if scenarioletter == 'Baseline':
        tabname = 'Baseline Detailed Inputs'
    else:
        tabname = 'Scenario ' + scenarioletter +' Detailed Inputs'

        
    scenariotitle = {'Baseline':'Baseline','A':'Low','B':'Medium','C':'High'}
    scenarioname = scenariotitle[scenarioletter]
    scenarioresults = {}
    pivotdictionary = {'ldv': 'Light Duty', 'mdv': 'Medium Duty', 'hdv':'Heavy Duty', 'GAS':'Gasoline',\
                       'DSL':'Diesel','ELC':'Electricity', 'NAG':'Fossil Natural Gas', 'HYD':'Hydrogen',\
                       'PHV': 'Plug-in Hybrid', 'pop':'sales', 'con':'Fuel Consumption', 'mpg':'Fuel Economy', 'vmt': 
                       'VMT', 'bus': 'Bus','JET': 'Jet Fuel', 'RNJ': 'Renewable Jet Fuel','ETH':'Ethanol',\
                       'RNG':'Renewable Natural Gas','RHD':'Renewable Hydrogen','RND':'Renewable Diesel','BID':'Biodiesel'}

    new_dic = dict([[v,k] for k,v in pivotdictionary.items()])


    fuelcondict = {'GAS':['Gasoline', 'Ethanol'], 'DSL':['Diesel', 'Renewable Diesel', 'Biodiesel'],\
                   'NAG':['Fossil Natural Gas', 'Renewable Natural Gas'],'ELC':['Electricity'],\
                   'PHV':['Gasoline', 'Ethanol','Diesel', 'Renewable Diesel', 'Biodiesel','Electricity'], \
                   'HYD':['Hydrogen', 'Renewable Hydrogen']}

    vehicleclasses = ['ldv', 'mdv','hdv','bus']
    fueltypes = ['GAS', 'DSL', 'NAG', 'HYD', 'ELC', 'PHV']

    eVMT = wb.sheets[tabname].range('B135').options(pd.DataFrame, expand = 'table').value
    eVMT.columns = calyr
    eVMT.index = ['ldv', 'mdv', 'hdv','bus']
    scenarioer = {}

    modyr = [i for i in range(1961,2036)]
    calyr = [i for i in range(2019,2036)]
    fuelci = pd.DataFrame(columns = calyr).fillna(0)

    scenarioer['GAS'] = pd.DataFrame(index = modyr, columns = calyr).fillna(value = 100.82)
    scenarioer['GAS'].index.name = 'CARBOB CI [gCO2e/MJ]'

    scenarioer['DSL'] = pd.DataFrame(index = modyr, columns = calyr).fillna(value = 100.45)
    scenarioer['DSL'].index.name = 'ULSD CI [gCO2e/MJ]'

    scenarioer['JET'] = pd.DataFrame(index = modyr, columns = calyr).fillna(value = 89.37)
    scenarioer['JET'].index.name = 'Jet Fuel CI [gCO2e/MJ]'

    scenarioer['NAG'] = pd.DataFrame(index = modyr, columns = calyr).fillna(value = 79.21)
    scenarioer['NAG'].index.name = 'Natural Gas CI [gCO2e/MJ]'

    scenarioer['HYD'] = pd.DataFrame(index = modyr, columns = calyr).fillna(value = 117.67)
    scenarioer['HYD'].index.name = 'Hydrogen CI [gCO2e/MJ]'

    scenarioer['ETH'] = pd.DataFrame(index = modyr, columns = calyr).fillna(1)
    scenarioer['ETH'].index.name = 'Ethanol CI [gCO2e/MJ]'
    fuelci.loc['Ethanol'] = wb.sheets[tabname].range('C10:S10').value
    scenarioer['ETH'] = scenarioer['ETH'] * fuelci.loc['Ethanol']

    scenarioer['BID'] = pd.DataFrame(index = modyr, columns = calyr).fillna(1)
    scenarioer['BID'].index.name = 'Biodiesel CI [gCO2e/MJ]'
    fuelci.loc['Biodiesel'] = wb.sheets[tabname].range('C25:S25').value
    scenarioer['BID'] = scenarioer['BID'] * fuelci.loc['Biodiesel']

    scenarioer['RND'] = pd.DataFrame(index = modyr, columns = calyr).fillna(1)
    scenarioer['RND'].index.name = 'Renewable Diesel CI [gCO2e/MJ]'
    fuelci.loc['Renewable Diesel'] = wb.sheets[tabname].range('C38:S38').value
    scenarioer['RND'] = scenarioer['RND'] * fuelci.loc['Renewable Diesel']

    scenarioer['RNJ'] = pd.DataFrame(index = modyr, columns = calyr).fillna(1)
    scenarioer['RNJ'].index.name = 'Renewable Jet Fuel CI [gCO2e/MJ]'
    fuelci.loc['Renewable Diesel'] = wb.sheets[tabname].range('C38:S38').value
    scenarioer['RNJ'] = scenarioer['RNJ'] * fuelci.loc['Renewable Diesel']

    scenarioer['RNG'] = pd.DataFrame(index = modyr, columns = calyr).fillna(1)
    scenarioer['RNG'].index.name = 'Renewable Natural Gas CI [gCO2e/MJ]'
    fuelci.loc['Renewable Natural Gas'] = wb.sheets[tabname].range('C53:S53').value
    scenarioer['RNG'] = scenarioer['RNG'] * fuelci.loc['Renewable Natural Gas']

    scenarioer['RHD'] = pd.DataFrame(index = modyr, columns = calyr).fillna(1)
    scenarioer['RHD'].index.name = 'Renewable Hydrogen CI [gCO2e/MJ]'
    fuelci.loc['Hydrogen'] = wb.sheets[tabname].range('C62:S62').value
    scenarioer['RHD'] = scenarioer['RHD'] * fuelci.loc['Hydrogen']

    scenarioer['ELC'] = pd.DataFrame(index = modyr, columns = calyr).fillna(1)
    scenarioer['ELC'].index.name = 'Electricity CI [gCO2e/MJ]'
    fuelci.loc['Electricity'] = wb.sheets[tabname].range('C66:S66').value
    scenarioer['ELC'] = scenarioer['ELC'] * fuelci.loc['Electricity']

    cwd = os.getcwd()
    start = time.time()

    file = cwd +"\\PKL Files\\baseline.pkl"
    pkl_file = open(file,'rb')
    mydict = pickle.load(pkl_file)
    pkl_file.close()
    baseline = copy.deepcopy(mydict)
    scenario = copy.deepcopy(mydict)

    file = cwd +"\\PKL Files\\baselineresults.pkl"
    pkl_file = open(file,'rb')
    mydict = pickle.load(pkl_file)
    pkl_file.close()
    baselineresults = copy.deepcopy(mydict)

    # file = cwd +"\\PKL Files\\costdata.pkl"
    # pkl_file = open(file,'rb')
    # mydict = pickle.load(pkl_file)
    # pkl_file.close()
    # costdata = copy.deepcopy(mydict)

    #update VMT
    vmtreduction = wb.sheets[tabname].range('B141').options(pd.DataFrame, expand = 'table').value

    for key in vehicleclasses:
        for c in vmtreduction.columns.tolist():
            scenario[key]['vmt'][c] *= (1-vmtreduction.loc[pivotdictionary[key],c])


    #update sales
    vehicleclasses = ['ldv','mdv','hdv','bus']
    scenariosales = wb.sheets[tabname].range('B83').options(pd.DataFrame, expand = 'table').value
    for key in vehicleclasses:
        for fuel in fueltypes:
            for c in range(2020,2036):
                scenario[key][fuel+'pop'].loc[c] = baseline[key]['TOTpop'].loc[c] * scenariosales.loc[key.upper()+fuel,c]

    #update fuel economy
    fueleconomycat = ['GAS', 'DSL']
    scenariompgimprovement = wb.sheets[tabname].range('B109').options(pd.DataFrame, expand = 'table').value

    for key in vehicleclasses:
        for fuel in fueleconomycat:
            for c in range(2019,2036):
                scenario[key][fuel+'mpg'].loc[c] *= (1+scenariompgimprovement.loc[key.upper()+fuel,c])
                scenario[key][fuel + 'mpg'].loc[c+1,c] = scenario[key][fuel + 'mpg'].loc[c,c]



    #fuel economy in pivot table format
    mpgpivotdictionary = {}
    x = 1
    for key in vehicleclasses:
        for fuel in fueltypes:
            if fuel != "PHV":
                temp = copy.deepcopy(scenario[key][fuel + 'mpg'].fillna(0))
                old = list(temp.columns)
                temp.columns = [int(x) for x in old]
                temp['Fuel Type'] = pivotdictionary[fuel]
                temp['Vehicle Class'] = pivotdictionary[key]
                temp.index.name = 'Model Year'
                mpgpivotdictionary[str(x)] = temp.fillna(0)
                x += 1

    d = { k: v.reset_index(level = 0, inplace = True) for k, v in mpgpivotdictionary.items()}
    d = { k: v.set_index(['Model Year', 'Fuel Type', 'Vehicle Class']) for k, v in mpgpivotdictionary.items()}
    mpgpivottable = pd.concat(d.values(), axis=0, sort = False)

    scenarioresults['Fuel Economy'] = mpgpivottable

    # #For reference. New vehicle MPG
    listofyears = list(range(2019,2036))
    test = copy.deepcopy(mpgpivottable[listofyears])
    test.reset_index(inplace = True)
    test = test.melt(id_vars = ['Model Year','Fuel Type', 'Vehicle Class'], var_name = ['Calendar Year'])
    test = test.fillna(0)
    test['Model Year'] = test['Model Year'].astype(int)
    test['Calendar Year'] = test['Calendar Year'].astype(int)
    test2 = test[test['Model Year'] == test['Calendar Year']]
    test2 = test2.pivot_table(index=['Vehicle Class', 'Fuel Type'],columns = 'Calendar Year', values = 'value').fillna(0)
    # test2.loc[('Gasoline', 'Heavy Duty')] *= gge_to_dge
    # test2.loc[('Gasoline', 'Medium Duty')] *= gge_to_dge
    # test2.loc[('Gasoline', 'Light Duty')] *= gge_to_dge
    # test2[test2.T.plot()
    # test2.to_excel(graphfolder + "\\Testing\\baselinempg.xlsx")
    toplot = test2.loc['Heavy Duty'][list(range(2019,2036))]
    toplot = toplot.drop('Gasoline')
    ax = toplot.T.plot()
    ax.grid()
    ax.set_axisbelow(True)
    ax.set_xlim(2019,2040)
    ax.set_xticks(range(2019,2036))
    ax.tick_params(axis = 'x', labelrotation = 90)
    ax.legend(title = None, bbox_to_anchor=(1, 0.8), loc = 'upper left')
    ax.set_ylabel('Fuel Economy [m/dge]')
    ax.set_xlabel('Year')
    fig = ax.get_figure()
    fig.savefig(graphfolder + "\\Testing\\HDfueleconomy.png",bbox_inches='tight')
    test1 = toplot
    test1.index.name = 'm/dge'
    test1.to_excel(graphfolder + "\\Testing\\heavydutyfueleconomy.xlsx")

    tests = 1/test1.loc['Electricity']
    tests = tests * (gge_to_MJ/kWh_to_MJ * test2)

    toplot = test2.loc['Light Duty'][list(range(2019,2036))]
    toplot = toplot.drop('Diesel')
    toplot *= gge_to_dge
    order = ['Gasoline','Electricity','Hydrogen','Fossil Natural Gas']
    ax = toplot.loc[order].T.plot()#color = ['b','tab:orange','g','r'])
    ax.grid()
    ax.set_axisbelow(True)
    ax.set_xlim(2019,2035)
    ax.set_xticks(range(2019,2036))
    plt.setp(ax.lines[0], marker = 'x', markevery = 2)
    ax.tick_params(axis = 'x', labelrotation = 90)
    ax.legend(title = None, bbox_to_anchor=(1, 0.8), loc = 'upper left')
    ax.set_ylabel('Fuel Economy [m/gge]')
    ax.set_xlabel('Year')
    fig = ax.get_figure()
    fig.savefig(graphfolder + "\\Testing\\LDfueleconomy.png",bbox_inches='tight')
    test = toplot.loc[order]
    test.index.name = 'm/gge'
    test.to_excel(graphfolder + "\\Testing\\lightdutyfueleconomy.xlsx")

    # toplot = test2.loc['Medium Duty'][list(range(2019,2036))]
    # toplot = toplot.drop('Diesel')
    # toplot *= gge_to_dge
    # order = ['Gasoline','Electricity','Hydrogen','Fossil Natural Gas']
    # test = toplot.loc[order]
    # test.index.name = 'm/gge'
    # test.to_excel(graphfolder + "\\Testing\\mediumdutygasfueleconomy.xlsx")

    # toplot = test2.loc['Medium Duty'][list(range(2019,2036))]
    # toplot = toplot.drop('Gasoline')
    # order = ['Diesel','Electricity','Hydrogen','Fossil Natural Gas']
    # test = toplot.loc[order]
    # test.index.name = 'm/dge'
    # test.to_excel(graphfolder + "\\Testing\\mediumdutydslfueleconomy.xlsx")

    # toplot = test2.loc['Bus'][list(range(2019,2036))]
    # toplot = toplot.drop('Gasoline')
    # test1 = toplot
    # test1.index.name = 'm/dge'
    # test1.to_excel(graphfolder + "\\Testing\\busfueleconomy.xlsx")

    # weightedavgmpg = (scenario['hdv']['DSLmpg']*scenario['hdv']['DSLpop'])/scenario['hdv']['DSLpop'].sum()
    # weightedavgmpg.mean().to_excel(graphfolder + "\\Testing\\DSLweightedavg_mpg.xlsx")

    #put into pivot table format
    poppivotdictionary = {}

    x = 1
    for key in vehicleclasses:
    #     print(key)
        for fuel in fueltypes:
            temp = copy.deepcopy(scenario[key][fuel + 'pop'].fillna(0))
            old = list(temp.columns)
            temp.columns = [int(x) for x in old]
            temp['Fuel Type'] = pivotdictionary[fuel]
            temp['Vehicle Class'] = pivotdictionary[key]
            temp.index.name = 'Model Year'
            poppivotdictionary[str(x)] = temp.fillna(0)
            x += 1

    # print('preend', time.time() -start)
    d = { k: v.reset_index(level = 0, inplace = True) for k, v in poppivotdictionary.items()}
    d = { k: v.set_index(['Model Year', 'Fuel Type', 'Vehicle Class']) for k, v in poppivotdictionary.items()}
    populationpivottable = pd.concat(d.values(), axis=0, sort = False)
    # print('fakeend', time.time()-start) 

    populationpivottable = populationpivottable.reindex(columns=sorted(populationpivottable.columns))
    vmt = copy.deepcopy(populationpivottable)
    scenarioresults['Population'] = copy.deepcopy(populationpivottable)

    listofyears = [x for x in range(2019,2036)]
    order = ['Gasoline', 'Diesel', 'Fossil Natural Gas', 'Electricity', 'Plug-in Hybrid', 'Hydrogen']
    order2 = ['Light Duty', 'Medium Duty', 'Heavy Duty', 'Bus']

    # for c in range(2019,2036):
    #     populationpivottable.rename(columns = {c:str(c)}, inplace = True)

    #new population graphs
    tograph = ['Fuel Type']
    vehicleclass = ['Light Duty', 'Medium Duty', 'Heavy Duty', 'Bus']
    if scenarioletter == 'Baseline':
        savetitle = 'Scenario ' + scenarioletter +' Vehicle Population By Vehicle Class'
        figtitle = scenarioname + '\n'+ 'Vehicle Population By Vehicle Class'
        population = populationpivottable[listofyears].groupby(['Vehicle Class'], sort = False).sum()
        # ax = population.sort_values(('2016'), ascending = False).T.plot(kind = 'bar', stacked = True, title = figtitle)
        ax = population.loc[order2].T.plot(kind = 'bar', stacked = True, title = figtitle)
        ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
        ax.grid()
        ax.set_axisbelow(True)
        ax.set_ylabel('Vehicle Population [vehicles]')
        ax.set_xlabel('Year')
        fig = ax.get_figure();
        fig.savefig(graphfolder +"\\Baseline\\Population\\" +  savetitle + ".jpg",bbox_inches='tight')
        population.loc[order2].to_excel(graphfolder +"\\Baseline\\Population\\" +  savetitle + ".xlsx")

    #vmt calc
    vmt[vmt.index.get_level_values(level = 'Vehicle Class').isin(['Light Duty'])] *= scenario['ldv']['vmt'].mean()
    vmt[vmt.index.get_level_values(level = 'Vehicle Class').isin(['Medium Duty'])] *= scenario['mdv']['vmt'].mean()
    vmt[vmt.index.get_level_values(level = 'Vehicle Class').isin(['Heavy Duty'])] *= scenario['hdv']['vmt'].mean()
    vmt[vmt.index.get_level_values(level = 'Vehicle Class').isin(['Bus'])] *= scenario['bus']['vmt'].mean()

    # toprint = vmt.groupby(['Vehicle Class']).sum()
    # toprint.loc['Total'] = toprint.sum()
    # toprint.index.name = 'thousands of miles'
    # toprint.to_excel(graphfolder + "\\Testing\\vmtbyclass.xlsx")
    scenarioresults['VMT'] = vmt

    tograph = ['Fuel Type']
    vehicleclass = ['Light Duty', 'Medium Duty', 'Heavy Duty', 'Bus']


    savetitle = 'Scenario ' + scenarioletter +' VMT By Vehicle Class'
    figtitle = scenarioname + '\n'+ 'VMT By Vehicle Class'
    vmtpivot = vmt[listofyears].groupby(['Vehicle Class'], sort = False).sum()
    ax = vmtpivot.loc[order2].T.plot(kind = 'bar', stacked = True, title = figtitle)
    if scenarioname != "Baseline":
        reference = baselineresults['VMT'].sum()
        ax.axhline(y = reference[2019], xmin=0, xmax=(2019-2019+.75)/(2036-2019), c = 'black', label = 'Baseline')
        for c in range(2019,2036):
            x_min = (c-2019+.25)/(2036-2019)
            x_max = (c-2019+.75)/(2036-2019)
            ax.axhline(y = reference[c], xmin=x_min, xmax=x_max, c = 'black')
    ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
    ax.grid()
    ax.set_axisbelow(True)
    ax.set_ylabel('VMT [miles]')
    ax.set_xlabel('Year')
    fig = ax.get_figure()
    fig.savefig(graphfolder +"\\Baseline\\VMT\\" +  savetitle + ".jpg",bbox_inches='tight')
    vmtpivot.loc[order2].to_excel(graphfolder +"\\Baseline\\VMT\\" +  savetitle + ".xlsx")
    plt.close('all')

    for thing in tograph:
        for vehclass in vehicleclass:
            figtitle = scenarioname + ' ' + vehclass + '\n'+' VMT by ' + thing
            savetitle = 'Scenario ' + scenarioletter +' ' + vehclass +' VMT by ' + thing
            vmtpivot = vmt[listofyears].groupby(['Vehicle Class',thing], sort = False).sum()
            toplot = vmtpivot.loc[vehclass]
            ax = toplot.loc[order].T.plot(kind = 'bar', stacked = True, title = figtitle)
            if scenarioname != "Baseline":
                reference = baselineresults['VMT'].groupby(['Vehicle Class']).sum().loc[vehclass]
                ax.axhline(y = reference[2019], xmin=0, xmax=(2019-2019+.75)/(2036-2019), c = 'black', label = 'Baseline')
                for c in range(2019,2036):
                    x_min = (c-2019+.25)/(2036-2019)
                    x_max = (c-2019+.75)/(2036-2019)
                    ax.axhline(y = reference[c], xmin=x_min, xmax=x_max, c = 'black')
            ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
            ax.grid()
            ax.set_axisbelow(True)
            ax.set_ylabel('VMT [miles]')
            ax.set_xlabel('Year')
            fig = ax.get_figure()
            fig.savefig(graphfolder +"\\" + scenarioname  +"\\VMT\\" +  savetitle + ".jpg",bbox_inches='tight')
            toplot.loc[order].to_excel(graphfolder +"\\" + scenarioname  +"\\VMT\\" +  savetitle + ".xlsx")



    for thing in tograph:
        for vehclass in vehicleclass:
            figtitle = scenarioname + ' ' + vehclass + '\n'+' Vehicle Population by ' + thing
            savetitle = 'Scenario ' + scenarioletter +' ' + vehclass +' Vehicle Population by ' + thing
            population = populationpivottable[listofyears].groupby(['Vehicle Class',thing], sort = False).sum()
            toplot = population.loc[vehclass]
            ax = toplot.loc[order].T.plot(kind = 'bar', stacked = True, title = figtitle)
            ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
            ax.grid()
            ax.set_axisbelow(True)
            ax.set_ylabel('Vehicle Population [vehicles]')
            ax.set_xlabel('Year')
            fig = ax.get_figure();
            fig.savefig(graphfolder +"\\" + scenarioname  +"\\Population\\" +  savetitle + ".jpg",bbox_inches='tight')
            toplot.loc[order].to_excel(graphfolder +"\\" + scenarioname  +"\\Population\\" +  savetitle + ".xlsx")

    salespivottable = copy.deepcopy(populationpivottable)
    salespivottable = salespivottable.diff(axis = 1).fillna(0)
    salespivottable = salespivottable[salespivottable>0].fillna(0)
    salespivottable.reset_index(inplace = True)
    test = salespivottable.melt(id_vars = ['Model Year','Fuel Type', 'Vehicle Class'], var_name = ['Calendar Year'])
    test = test.fillna(0)
    test['Model Year'] = test['Model Year'].astype(int)
    test['Calendar Year'] = test['Calendar Year'].astype(int)
    test2 = test[test['Model Year'] >= test['Calendar Year']]
    test2 = test2.pivot_table(index=['Model Year', 'Fuel Type', 'Vehicle Class'],columns = 'Calendar Year', values = 'value').fillna(0)
    salespivottable = test2
    salespivottable[2020] = salespivottable[2021]*.99
    salespivottable[2019] = salespivottable[2020]*.99

    #new sales graphs
    tograph = ['Fuel Type']
    vehicleclass = ['Light Duty', 'Medium Duty', 'Heavy Duty', 'Bus']

    if scenarioletter == 'Baseline':
        savetitle = 'Scenario ' + scenarioletter +' Vehicle sales By Vehicle Class'
        figtitle = scenarioname + '\n'+ 'Vehicle Sales By Vehicle Class'
        sales = salespivottable[listofyears].groupby(['Vehicle Class'], sort = False).sum()
        ax = sales.loc[order2].T.plot(kind = 'bar', stacked = True, title = figtitle)
        ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
        ax.grid()
        ax.set_axisbelow(True)
        ax.set_ylabel('Vehicle sales [vehicles]')
        ax.set_xlabel('Year')
        fig = ax.get_figure()
        fig.savefig(graphfolder +"\\Baseline\\Sales\\" +  savetitle + ".jpg",bbox_inches='tight')
        sales.loc[order2].to_excel(graphfolder +"\\Baseline\\Sales\\" +  savetitle + ".xlsx")


    for thing in tograph:
        for vehclass in vehicleclass:
            figtitle = scenarioname + ' ' + vehclass + '\n'+' Vehicle Sales by ' + thing
            savetitle = 'Scenario ' + scenarioletter +' ' + vehclass +' Vehicle Sales by ' + thing
            sales = salespivottable[listofyears].groupby(['Vehicle Class',thing], sort = False).sum()
            toplot = sales.loc[vehclass]
            ax = toplot.loc[order].T.plot(kind = 'bar', stacked = True, title = figtitle)
            ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
            ax.grid()
            ax.set_axisbelow(True)
            ax.set_ylabel('Vehicle Sales [vehicles]')
            ax.set_xlabel('Year')
            fig = ax.get_figure()
            fig.savefig(graphfolder +"\\" + scenarioname  +"\\Sales\\" +  savetitle + ".jpg",bbox_inches='tight')
            toplot.loc[order].to_excel(graphfolder +"\\" + scenarioname  +"\\Sales\\" +  savetitle + ".xlsx")

    end = time.time()

    #fuel consumption
    fuelcon = {}
    afvblend = wb.sheets[tabname].range('B69').options(pd.DataFrame, expand = 'table').value
    afvblend.columns = range(2019,2036)

    for key in vehicleclasses:
        fuelcon[key] = {}
        for fuel in fueltypes:
            fuelcon[key][fuel] = {}
            for itemname in fuelcondict[fuel]:
                fuelcon[key][fuel][itemname] = pd.DataFrame(index = range(1961,2036), columns = calyr).fillna(0)


    for key in vehicleclasses:
        for fuel in fueltypes:
            if fuel == 'PHV':
                elcconsumption = (scenario[key]['vmt'] * eVMT.loc[key]*\
                                  scenario[key][fuel + 'pop'])/scenario[key]['ELCmpg'][scenario[key]['ELCmpg']>0]
                fuelcon[key][fuel]['Electricity'] = elcconsumption
                if key == 'ldv' or key == 'mdv':
                    gasconsumption = (scenario[key]['vmt'] * (1-eVMT.loc[key])*\
                                  scenario[key][fuel + 'pop'])/scenario[key]['GASmpg'][scenario[key]['GASmpg']>0]
                    gasconsumption = gasconsumption.fillna(0)
                    gasoline = copy.deepcopy(gasconsumption) * (1-afvblend.loc['Ethanol Scenario Blend'])
                    ethanol = copy.deepcopy(gasconsumption) * afvblend.loc['Ethanol Scenario Blend']
                    fuelcon[key][fuel]['Gasoline'] = gasoline
                    fuelcon[key][fuel]['Ethanol'] = ethanol
                else:
                    gasconsumption = (scenario[key]['vmt'] * (1-eVMT.loc[key])*\
                                  scenario[key][fuel + 'pop'])/scenario[key]['DSLmpg'][scenario[key]['DSLmpg']>0]
                    gasconsumption = gasconsumption.fillna(0)
                    renewablediesel = copy.deepcopy(gasconsumption) * afvblend.loc['Renewable Diesel Scenario Blend']
                    biodiesel = copy.deepcopy(gasconsumption) * afvblend.loc['Biodiesel Scenario Blend']
                    diesel = copy.deepcopy(gasconsumption) * afvblend.loc['Fossil Diesel Scenario Blend']
                    fuelcon[key][fuel]['Renewable Diesel'] = renewablediesel
                    fuelcon[key][fuel]['Biodiesel'] = biodiesel
                    fuelcon[key][fuel]['Diesel'] = diesel
            elif fuel == 'HYD':
                hydrogenconsumption = scenario[key]['vmt'] * scenario[key][fuel + 'pop']/scenario[key][fuel + 'mpg'][scenario[key][fuel + 'mpg']>0]
                fuelcon[key]['HYD']['Hydrogen'] = copy.deepcopy(hydrogenconsumption) * (1-afvblend.loc['Renewable Hydrogen Scenario Blend'])
                fuelcon[key]['HYD']['Renewable Hydrogen'] = copy.deepcopy(hydrogenconsumption) * afvblend.loc['Renewable Hydrogen Scenario Blend']
            elif fuel == 'GAS':
                gasconsumption = scenario[key]['vmt'] * scenario[key][fuel + 'pop']/scenario[key][fuel + 'mpg'][scenario[key][fuel + 'mpg']>0]
                fuelcon[key]['GAS']['Gasoline'] = copy.deepcopy(gasconsumption) * (1-afvblend.loc['Ethanol Scenario Blend'])
                fuelcon[key]['GAS']['Ethanol'] = copy.deepcopy(gasconsumption) * afvblend.loc['Ethanol Scenario Blend']
            elif fuel == 'DSL':
                dslconsumption = scenario[key]['vmt'] * scenario[key][fuel + 'pop']/scenario[key][fuel + 'mpg'][scenario[key][fuel + 'mpg']>0]
    #             print(dslconsumption)
                renewabledieselconsumption = copy.deepcopy(dslconsumption) * afvblend.loc['Renewable Diesel Scenario Blend']
                biodieselconsumption = copy.deepcopy(dslconsumption) * afvblend.loc['Biodiesel Scenario Blend']
                dieselconsumption = copy.deepcopy(dslconsumption) * (1-afvblend.loc['Renewable Diesel Scenario Blend'] - afvblend.loc['Biodiesel Scenario Blend'])
                fuelcon[key]['DSL']['Diesel'] = dieselconsumption
                fuelcon[key]['DSL']['Renewable Diesel'] = renewabledieselconsumption
                fuelcon[key]['DSL']['Biodiesel'] = biodieselconsumption
            elif fuel == 'NAG':
                nagconsumption = scenario[key]['vmt'] * scenario[key][fuel + 'pop']/scenario[key][fuel + 'mpg'][scenario[key][fuel + 'mpg']>0]
                fuelcon[key]['NAG']['Fossil Natural Gas'] = copy.deepcopy(nagconsumption) * (1-afvblend.loc['RNG Scenario Blend'])
                fuelcon[key]['NAG']['Renewable Natural Gas'] = copy.deepcopy(nagconsumption) * afvblend.loc['RNG Scenario Blend']
            elif fuel == 'ELC':
                elcconsumption= scenario[key]['vmt'] * scenario[key][fuel + 'pop']/scenario[key][fuel + 'mpg'][scenario[key][fuel + 'mpg']>0]
                fuelcon[key]['ELC']['Electricity'] = elcconsumption

    #fuel consumption pivot table
    fuelconpivotdictionary = {}

    x = 1
    for key in vehicleclasses:
        for fuel in fueltypes:
            for itemname in fuelcon[key][fuel]:
                temp = copy.deepcopy(fuelcon[key][fuel][itemname].fillna(0))
                old = list(temp.columns)
                temp.columns = [int(x) for x in old]
                temp['Fuel Type'] = pivotdictionary[fuel]
                temp['Vehicle Class'] = pivotdictionary[key]
                temp['Detailed Fuel Type'] = itemname
                temp['On or Off'] = 'On'
                temp.index.name = 'Model Year'
                fuelconpivotdictionary[str(x)] = temp.fillna(0)
                x += 1


    d = { k: v.reset_index(level = 0, inplace = True) for k, v in fuelconpivotdictionary.items()}
    d = { k: v.set_index(['On or Off','Model Year', 'Fuel Type', 'Detailed Fuel Type','Vehicle Class']) for k, v in fuelconpivotdictionary.items()}
    fuelpivottable = pd.concat(d.values(), axis=0, sort = False)

    fuelpivottable.index.name = 'DGE'

    #add offroad fuel consumption
    offroadappend = {}
    x = 1

    offroad = wb.sheets[tabname].range('B147').options(pd.DataFrame, expand = 'table').value
    offroadrenewable = wb.sheets[tabname].range('B152').options(pd.DataFrame, expand = 'table').value

    for offtype in offroad.index:
        temp = copy.deepcopy(offroad.loc[[offtype]]) * (1-copy.deepcopy(offroadrenewable.loc[offtype + ' (% renewable)']))
        if offtype == 'Aviation':
            fuel = 'Jet Fuel'
        else:
            fuel = 'Diesel'
        temp['Fuel Type'] = fuel
        temp['Vehicle Class'] = offtype
        temp['Detailed Fuel Type'] = fuel
        temp['On or Off'] = 'Off'
        temp.index.name = 'Model Year'
        offroadappend[str(x)] = temp.fillna(0)
        x += 1

    for offtype in offroad.index:
        temp = copy.deepcopy(offroad.loc[[offtype]]) * (copy.deepcopy(offroadrenewable.loc[offtype + ' (% renewable)']))
        if offtype == 'Aviation':
            fuel = 'Jet Fuel'
        else:
            fuel = 'Diesel'
        temp['Fuel Type'] = fuel
        temp['Vehicle Class'] = offtype
        temp['Detailed Fuel Type'] = 'Renewable ' + fuel
        temp['On or Off'] = 'Off'
        temp.index.name = 'Model Year'
        offroadappend[str(x)] = temp.fillna(0)
        x += 1


    d = { k: v.reset_index(level = 0, inplace = True) for k, v in offroadappend.items()}
    d = { k: v.set_index(['On or Off','Model Year', 'Fuel Type', 'Detailed Fuel Type','Vehicle Class']) for k, v in offroadappend.items()}
    offroad = pd.concat(d.values(), axis=0, sort = False)

    fuelpivottable = fuelpivottable.append(offroad)

    #new fuel consumption graphs
    scenarioresults['Fuel Consumption'] = fuelpivottable

    categorytitle = {'ldv': 'Light Duty', 'mdv': 'Medium Duty', 'hdv': 'Heavy Duty', 'Total': 'Total'}

    figtitle = scenarioname + ' ' + '\n'+ 'Fuel Consumption By Vehicle Class'
    savetitle = 'Scenario ' + scenarioletter +' Fuel Consumption By Vehicle Class'
    fuelcons = fuelpivottable.loc['On',listofyears].groupby(['Vehicle Class']).sum()
    ax = fuelcons.sort_values((2019), ascending = False).T.plot(kind = 'bar', stacked = True, title = figtitle)
    ax.grid()
    ax.set_axisbelow(True)
    if scenarioname != "Baseline":
        reference = baselineresults['Fuel Consumption'].loc['On'].sum()
        ax.axhline(y = reference[2019], xmin=0, xmax=(2019-2019+.75)/(2036-2019), c = 'black', label = 'Baseline')
        for c in range(2019,2036):
            x_min = (c-2019+.25)/(2036-2019)
            x_max = (c-2019+.75)/(2036-2019)
            ax.axhline(y = reference[c], xmin=x_min, xmax=x_max, c = 'black')
    ax.set_ylabel('Fuel Consumption')
    ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
    ax.set_xlabel('Year')
    fig = ax.get_figure()
    fig.savefig(graphfolder +"\\" + scenarioname  +"\\Fuel\\" +  savetitle + ".jpg",bbox_inches='tight')
    fuelcons.sort_values((2019), ascending = False).to_excel(graphfolder +"\\" + scenarioname  +"\\Fuel\\" +  savetitle + ".xlsx")

    tograph = ['Detailed Fuel Type']
    for thing in tograph:
        for vehclass in vehicleclass:
            figtitle = scenarioname + ' ' + vehclass + '\n'+' Fuel Consumption by ' + thing
            savetitle = 'Scenario ' + scenarioletter +' ' + vehclass +' Fuel Consumption by ' + thing
            fuel = fuelpivottable.loc['On',listofyears].groupby(['Vehicle Class',thing], sort = False).sum()
            toplot = fuel.loc[vehclass]
            ax = toplot.T.plot(kind = 'bar', stacked = True, title = figtitle)
            if scenarioname != "Baseline":
                reference = baselineresults['Fuel Consumption'].groupby(['Vehicle Class']).sum().loc[vehclass]
                ax.axhline(y = reference[2019], xmin=0, xmax=(2019-2019+.75)/(2036-2019), c = 'black', label = 'Baseline')
                for c in range(2019,2036):
                    x_min = (c-2019+.25)/(2036-2019)
                    x_max = (c-2019+.75)/(2036-2019)
                    ax.axhline(y = reference[c], xmin=x_min, xmax=x_max, c = 'black')
            ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
            ax.grid()
            ax.set_axisbelow(True)
            ax.set_ylabel('Fuel Consumption')
            ax.set_xlabel('Year')
            fig = ax.get_figure()
            fig.savefig(graphfolder +"\\" + scenarioname  +"\\Fuel\\" +  savetitle + ".jpg",bbox_inches='tight')
            toplot.to_excel(graphfolder +"\\" + scenarioname  +"\\Fuel\\" +  savetitle + ".xlsx")


    figtitle = scenarioname + ' On and Offroad'+'\n'+' Fuel Consumption'
    savetitle = 'Scenario ' + scenarioletter + ' On and Offroad Fuel Consumption'
    fuel = fuelpivottable[listofyears].groupby('Vehicle Class').sum()
    ax = fuel.T.plot(kind = 'bar', stacked = True, title = figtitle)
    if scenarioname != "Baseline":
        reference = baselineresults['Fuel Consumption'].loc[listofyears].sum()
        ax.axhline(y = reference[2019], xmin=0, xmax=(2019-2019+.75)/(2036-2019), c = 'black', label = 'Baseline')
        for c in range(2019,2036):
            x_min = (c-2019+.25)/(2036-2019)
            x_max = (c-2019+.75)/(2036-2019)
            ax.axhline(y = reference[c], xmin=x_min, xmax=x_max, c = 'black')
    ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
    ax.grid()
    ax.set_axisbelow(True)
    ax.set_ylabel('Fuel Consumption')
    ax.set_xlabel('Year')
    fig = ax.get_figure()
    fig.savefig(graphfolder +"\\" + scenarioname  +"\\Fuel\\" +  savetitle + ".jpg",bbox_inches='tight')
    toplot.to_excel(graphfolder +"\\" + scenarioname  +"\\Fuel\\" +  savetitle + ".xlsx")


    #CO calculations

    fuelcon_to_EF = {'Gasoline': 'GAS', 'Ethanol': 'ETH', 'Diesel': 'DSL', 'Fossil Natural Gas': 'NAG', 'Hydrogen': 'HYD'\
                    ,'Biodiesel': 'BID', 'Renewable Diesel': 'RND', 'Renewable Natural Gas': 'RNG',\
                     'Renewable Hydrogen': 'RHD', 'Electricity': 'ELC','Jet Fuel':'JET','Renewable Jet Fuel':'RNJ'}

    CO2pivottable = copy.deepcopy(fuelpivottable.fillna(0))
    CO2pivottable *= dge_to_MJ
    for fuel in CO2pivottable.index.get_level_values('Detailed Fuel Type').unique().tolist():
        filter1 = CO2pivottable.index.get_level_values('Detailed Fuel Type') == fuel
        CO2pivottable.loc[(filter1),:] *= scenarioer[fuelcon_to_EF[fuel]].mean()

    #new CO2 graphs
    tograph = ['Detailed Fuel Type']
    vehicleclass = ['Light Duty', 'Medium Duty', 'Heavy Duty']
    scenarioresults['CO2'] = CO2pivottable

    categorytitle = {'ldv': 'Light Duty', 'mdv': 'Medium Duty', 'hdv': 'Heavy Duty', 'Total': 'Total'}
    infraorder = ['Light Duty', 'Medium Duty','Heavy Duty','Bus','Off-highway','Rail','Aviation']
    infracolor = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink']#,'tab:gray','tab:olive','tab:cyan','peru']
    colormap = zip(infraorder,infracolor)
    colormap = dict(colormap)

    figtitle = scenarioname + ' ' + '\n'+ 'CO'+ r'$_2$' + 'e Emissions By Vehicle Class'
    savetitle = 'Scenario ' + scenarioletter +' CO2e Emissions By Vehicle Class'
    COemissions = CO2pivottable.loc['On',listofyears].groupby(['Vehicle Class']).sum()
    indexorder = [x for x in infraorder if x in COemissions.index]
    colors = [colormap[x] for x in colormap if x in indexorder]

    ax = COemissions.sort_values((2019), ascending = False).T.plot(kind = 'bar', stacked = True, title = figtitle, color = colors)
    ax.grid()
    ax.set_axisbelow(True)
    # ax.axhline(y = CO2target, c = 'black', label = '2050 GHG Target')
    ax.set_ylabel('CO' + r'$_2$' + 'e Emissions [grams]')
    ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
    ax.set_xlabel('Year')
    fig = ax.get_figure()
    fig.savefig(graphfolder +"\\" + scenarioname  +"\\Emissions\\" +  savetitle + ".jpg",bbox_inches='tight')
    COemissions.sort_values((2019), ascending = False).to_excel(graphfolder +"\\" + scenarioname  +"\\Emissions\\" +  savetitle + ".xlsx")

    figtitle = scenarioname + ' ' + '\n'+ 'CO'+ r'$_2$' + 'e Emissions By Fuel Type'
    savetitle = 'Scenario ' + scenarioletter +' CO2e Emissions By Fuel Type'
    COemissions = CO2pivottable.loc['On',listofyears].groupby(['Fuel Type']).sum()
    # COemissions /= 1000000000000
    ax = COemissions.loc[order].T.plot(kind = 'bar', stacked = True, title = figtitle)
    ax.grid()
    ax.set_axisbelow(True)
    # ax.axhline(y = CO2target, c = 'black', label = '2050 GHG Target')
    ax.set_ylabel('CO' + r'$_2$' + 'e Emissions [grams]')
    ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
    ax.set_xlabel('Year')
    fig = ax.get_figure()
    fig.savefig(graphfolder +"\\" + scenarioname  +"\\Emissions\\" +  savetitle + ".jpg",bbox_inches='tight')
    COemissions.loc[order].to_excel(graphfolder +"\\" + scenarioname  +"\\Emissions\\" +  savetitle + ".xlsx")

    for thing in tograph:
        for vehclass in vehicleclass:
            figtitle = scenarioname + ' ' + vehclass + '\n' + 'CO' + r'$_2$' + 'e Emissions By ' + thing
            savetitle = 'Scenario ' + scenarioletter +' ' + vehclass + ' CO2e Emissions By ' + thing
    #         reference = baselineresults['CO2'].groupby(['Vehicle Class'], sort = False).sum().loc[vehclass]
            COemissions = CO2pivottable.loc['On'].groupby(['Vehicle Class',thing], sort = False).sum()
            ax = COemissions[listofyears].loc[vehclass].T.plot(kind = 'bar', stacked = True, title = figtitle)
            ax.grid()
            ax.set_axisbelow(True)
            if scenarioname != "Baseline":
                ax.axhline(y = reference[2019], xmin=0, xmax=(2019-2019+.75)/(2036-2019), c = 'black', label = 'Baseline')
                for c in range(2020,2036):
                    x_min = (c-2019+.25)/(2036-2019)
                    x_max = (c-2019+.75)/(2036-2019)
                    ax.axhline(y = reference[c], xmin=x_min, xmax=x_max, c = 'black')
            ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
            ax.set_ylabel('CO' + r'$_2$' + 'e Emissions [grams]')
            ax.set_xlabel('Year')
            fig = ax.get_figure()
            fig.savefig(graphfolder +"\\" + scenarioname  +"\\Emissions\\" +  savetitle + ".jpg",bbox_inches='tight')
            COemissions[listofyears].to_excel(graphfolder +"\\" + scenarioname  +"\\Emissions\\" +  savetitle + ".xlsx")

    figtitle = scenarioname + ' On & Off Road' + '\n'+ 'CO'+ r'$_2$' + 'e Emissions By Fuel Type'
    savetitle = 'Scenario ' + scenarioletter +'On & Off Road CO2e Emissions By Fuel Type'
    COemissions = CO2pivottable[listofyears].groupby(['Vehicle Class']).sum()
    indexorder = [x for x in infraorder if x in COemissions.index]
    colors = [colormap[x] for x in colormap if x in indexorder]
    ax = COemissions.reindex(indexorder).T.plot(kind = 'bar', stacked = True, title = figtitle, color = colors)
    ax.grid()
    ax.set_axisbelow(True)
    # ax.axhline(y = CO2target, c = 'black', label = '2050 GHG Target')
    ax.set_ylabel('CO' + r'$_2$' + 'e Emissions [grams]')
    ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
    ax.set_xlabel('Year')
    fig = ax.get_figure()
    fig.savefig(graphfolder +"\\" + scenarioname  +"\\Emissions\\" +  savetitle + ".jpg",bbox_inches='tight')
    COemissions.to_excel(graphfolder +"\\" + scenarioname  +"\\Emissions\\" +  savetitle + ".xlsx")


    figtitle = scenarioname + ' Off Road' + '\n'+ 'CO'+ r'$_2$' + 'e Emissions By Fuel Type'
    savetitle = 'Scenario ' + scenarioletter +' Off Road CO2e Emissions By Fuel Type'
    COemissions = CO2pivottable.loc['Off',listofyears].groupby(['Vehicle Class']).sum()
    indexorder = [x for x in infraorder if x in COemissions.index]
    colors = [colormap[x] for x in colormap if x in indexorder]
    ax = COemissions.reindex(indexorder).T.plot(kind = 'bar', stacked = True, title = figtitle, color = colors)
    ax.grid()
    ax.set_axisbelow(True)
    # ax.axhline(y = CO2target, c = 'black', label = '2050 GHG Target')
    ax.set_ylabel('CO' + r'$_2$' + 'e Emissions [grams]')
    ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
    ax.set_xlabel('Year')
    fig = ax.get_figure()
    fig.savefig(graphfolder +"\\" + scenarioname  +"\\Emissions\\" +  savetitle + ".jpg",bbox_inches='tight')
    COemissions.to_excel(graphfolder +"\\" + scenarioname  +"\\Emissions\\" +  savetitle + ".xlsx")

    figtitle = scenarioname + ' Off Road CO'+ r'$_2$' + 'e' + '\n'+' Emissions By Fuel Type (excluding Aviation)'
    savetitle = 'Scenario ' + scenarioletter +'On & Off Road CO2e Emissions By Fuel Type (excluding Aviation)'
    COemissions = CO2pivottable[listofyears].groupby(['Vehicle Class']).sum()
    COemissions = COemissions.loc[['Light Duty','Medium Duty','Heavy Duty','Bus','Off-highway','Rail']]
    indexorder = [x for x in infraorder if x in COemissions.index]
    colors = [colormap[x] for x in colormap if x in indexorder]
    ax = COemissions.reindex(indexorder).T.plot(kind = 'bar', stacked = True, title = figtitle, color = colors)
    ax.grid()
    ax.set_axisbelow(True)
    # ax.axhline(y = CO2target, c = 'black', label = '2050 GHG Target')
    ax.set_ylabel('CO' + r'$_2$' + 'e Emissions [grams]')
    ax.legend(title = None, bbox_to_anchor=(1,0.8), loc = 'upper left')
    ax.set_xlabel('Year')
    fig = ax.get_figure()
    fig.savefig(graphfolder +"\\" + scenarioname  +"\\Emissions\\" +  savetitle + ".jpg",bbox_inches='tight')
    COemissions.to_excel(graphfolder +"\\" + scenarioname  +"\\Emissions\\" +  savetitle + ".xlsx")

    pkl_file = cwd +"\\PKL Files\\"+ scenarioletter + "results.pkl"
    output = open(pkl_file,'wb')
    pickle.dump(scenarioresults,output)
    output.close()
#     print("pickle saved")
    
    pkl_file = cwd +"\\PKL Files\\"+ scenarioletter + "ER.pkl"
    output = open(pkl_file,'wb')
    pickle.dump(scenarioer,output)
    output.close()
#     print("pickle saved")

    test = copy.deepcopy(scenarioresults['Fuel Consumption'][listofyears].groupby('Fuel Type').sum().loc['Gasoline'])
    test/= gge_to_dge
    test.T.to_excel(graphfolder +"\\Testing\\gasoline_consumption.xlsx")

    test = copy.deepcopy(scenarioresults['Fuel Consumption'].loc['On'][listofyears].groupby('Fuel Type').sum().loc['Diesel'])
    test.T.to_excel(graphfolder +"\\Testing\\diesel_consumption.xlsx")
    
#     print(scenarioletter, scenarioresults['Fuel Consumption'].loc['On'].groupby('Fuel Type').sum().loc['Fossil Natural Gas',2030])
    print("Scenario " + scenarioletter +  " Complete")
    plt.close('all')

In [37]:
# CI calculation
# plt.show()
# scenariotargets = {'Baseline': 10, 'A':10, 'B':15,'C':20}

def CI_credits(scenarioletter):
        scenariotitle = {'Baseline':'Baseline','A':'Low','B':'Medium','C':'High'}
        scenarioname = scenariotitle[scenarioletter]
        scenariotargets = {'Baseline': 10, 'A':10, 'B':15,'C':20}
        initial_target = scenariotargets[scenarioletter]
        CI_compliance = copy.deepcopy(CI_compliance_read)
        EER = copy.deepcopy(EER_read)
        CI_compliance *= initial_target/100

        file = cwd +"\\PKL Files\\"+ scenarioletter + "ER.pkl"
        pkl_file = open(file,'rb')
        mydict = pickle.load(pkl_file)
        pkl_file.close()
        scenarioer = copy.deepcopy(mydict)

        file = cwd +"\\PKL Files\\"+ scenarioletter + "results.pkl"
        pkl_file = open(file,'rb')
        mydict = pickle.load(pkl_file)
        pkl_file.close()
        scenarioresults = copy.deepcopy(mydict)

        fuelpivottable = copy.deepcopy(scenarioresults['Fuel Consumption'])

        CI_standard_linear = pd.DataFrame(columns = list(range(2019,2036)))
        CI_standard_linear.loc['Gasoline Compliance'] = (1-CI_compliance.loc['Linear'])*scenarioer['GAS'].loc[2019,2019]
        CI_standard_linear.loc['Diesel Compliance'] = (1-CI_compliance.loc['Linear'])*scenarioer['DSL'].loc[2019,2019]
        CI_standard_linear.loc['Jet Fuel Compliance'] = (1-CI_compliance.loc['Linear'])*scenarioer['JET'].loc[2019,2019]
        CI_standard_linear.index.name = ['Linear Compliance [g/MJ]']
        CI_standard_linear.to_excel(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\Linear_Standard.xlsx")

        CI_standard_backloaded = pd.DataFrame(columns = list(range(2019,2036)))
        CI_standard_backloaded.loc['Gasoline Compliance'] = (1-CI_compliance.loc['Back-Loaded'])*scenarioer['GAS'].loc[2019,2019]
        CI_standard_backloaded.loc['Diesel Compliance'] = (1-CI_compliance.loc['Back-Loaded'])*scenarioer['DSL'].loc[2019,2019]
        CI_standard_backloaded.loc['Jet Fuel Compliance'] = (1-CI_compliance.loc['Back-Loaded'])*scenarioer['JET'].loc[2019,2019]
        CI_standard_backloaded.index.name = ['Back-Loaded Compliance [g/MJ]']
        CI_standard_backloaded.to_excel(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\BackLoaded_Standard.xlsx")

        #backloaded
        gas_subs = ['Gasoline','Ethanol']
        ldgas_subs = ['Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen','Electricity']

        diesel_subs = ['Diesel','Renewable Diesel','Biodiesel']
        mdhd_diesel_subs = ['Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen','Electricity']

        gas_sub_table = pd.DataFrame(columns = listofyears)
        for fuel in gas_subs:
            gas_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                    loc[fuel])*dge_to_MJ*(CI_standard_backloaded.loc['Gasoline Compliance']\
                                                                          -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Gasoline'])

        for fuel in ldgas_subs:
            gas_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Vehicle Class','Detailed Fuel Type']).sum().\
                                                    loc['Light Duty'].loc[fuel])*dge_to_MJ*(CI_standard_backloaded.loc['Gasoline Compliance']\
                                                                                           -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Gasoline'])

        dsl_sub_table = pd.DataFrame(columns = listofyears)
        for fuel in diesel_subs:
            dsl_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                    loc[fuel])*dge_to_MJ*(CI_standard_backloaded.loc['Diesel Compliance']\
                                                                          -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Diesel'])

        for fuel in mdhd_diesel_subs:
            dsl_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Vehicle Class','Detailed Fuel Type']).sum().\
                                                    loc[['Medium Duty','Heavy Duty','Bus']].groupby('Detailed Fuel Type').sum()\
                                                    .loc[fuel])*dge_to_MJ*(CI_standard_backloaded.loc['Diesel Compliance']\
                                                                                           -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Diesel'])

        #backloaded credit_deficit
        gas_subs = ['Gasoline','Ethanol']
        ldgas_subs = ['Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen','Electricity']

        diesel_subs = ['Diesel','Renewable Diesel','Biodiesel']
        mdhd_diesel_subs = ['Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen','Electricity']

        gas_sub_table = pd.DataFrame(columns = listofyears)
        for fuel in gas_subs:
            gas_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                    loc[fuel])*dge_to_MJ*(CI_standard_backloaded.loc['Gasoline Compliance']\
                                                                          -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Gasoline'])

        for fuel in ldgas_subs:
            gas_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Vehicle Class','Detailed Fuel Type']).sum().\
                                                    loc['Light Duty'].loc[fuel])*dge_to_MJ*(CI_standard_backloaded.loc['Gasoline Compliance']\
                                                                                           -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Gasoline'])


        jet_sub_table = pd.DataFrame(columns = listofyears)

        jet_sub_table.loc['Renewable Jet Fuel'] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                loc['Renewable Jet Fuel'])*dge_to_MJ*(CI_standard_backloaded.loc['Jet Fuel Compliance']\
                                                                      -scenarioer['RNJ'].mean())



        credit_bank = pd.DataFrame(columns = listofyears)
        credit_bank.loc['Bank, Annual'] = gas_sub_table.sum() + dsl_sub_table.sum() + jet_sub_table.sum()
        credit_bank.loc['Bank, Cumulative'] =  credit_bank.loc['Bank, Annual'].cumsum()
        credit_bank /= 1000000

        fig = plt.figure()
        figtitle = scenarioname + ' Scenario Credit Bank' + '\n' + 'Backloaded ' + str(initial_target) + "%"
        savetitle = scenarioletter + ' Credit Bank, backloaded'
        ax = fig.gca(title = figtitle)
        ax.bar(credit_bank.T.index, credit_bank.loc['Bank, Annual'],color=(1.0, 0.0, 0.1, 1.0))
        ax.plot(credit_bank.T.index, credit_bank.loc['Bank, Cumulative'])
        ax.set_xlabel('Year')
        ax.set_ylabel('Credits')
        ax.grid()
        ax.set_axisbelow(True)
        ax.set_xticks(listofyears)
        ax.tick_params(axis = 'x', rotation = 90)
        ax.set_xlim(2019,2035)
        ax.legend(['Bank Cumulative', 'Bank, Annual'])
        fig.savefig(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".jpg",bbox_inches='tight')
        credit_bank.to_excel(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".xlsx")

        #backloaded credits generated
        credits_deficits = pd.concat([gas_sub_table,dsl_sub_table,jet_sub_table])
        credits_deficits.index.name = "Fuel"
        credits_deficits = credits_deficits.groupby('Fuel').sum()

        toprint = credits_deficits[credits_deficits >5]/1000000
        toprint = toprint.fillna(0)
        test = toprint.sum(axis = 1)
        for fuel in test.index:
            if test.loc[fuel] <= 1:
                toprint = toprint.drop(fuel)

        infraorder = ['Gasoline', 'Ethanol','Diesel','Renewable Diesel','Biodiesel','Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen', 'Electricity','Renewable Jet Fuel']
        infracolor = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink','tab:gray','tab:olive','tab:cyan','darkgreen']
        colormap = zip(infraorder,infracolor)
        colormap = dict(colormap)
        indexorder = [x for x in infraorder if x in toprint.index]
        colors = [colormap[x] for x in colormap if x in indexorder]

        figtitle = scenarioname + ' Scenario Credits Generated' + '\n' + 'Backloaded, ' + str(initial_target) + "%"
        savetitle = scenarioletter + ' Credits Generated (backloaded)'
        ax = toprint.reindex(indexorder).T.plot(kind = 'bar',stacked = True, title = figtitle, color = colors)
        ax.set_xlabel('Year')
        ax.set_ylabel('Credits')
        ax.legend(title = False, bbox_to_anchor=(1,0.8), loc = 'upper left')
        ax.grid()
        ax.set_axisbelow(True)
        fig= ax.get_figure()
        fig.savefig(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".jpg",bbox_inches='tight')
        credits_deficits.reindex(infraorder).to_excel(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".xlsx")


        #linear

        gas_subs = ['Gasoline','Ethanol']
        ldgas_subs = ['Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen','Electricity']

        diesel_subs = ['Diesel','Renewable Diesel','Biodiesel']
        mdhd_diesel_subs = ['Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen','Electricity']

        gas_sub_table = pd.DataFrame(columns = listofyears)
        for fuel in gas_subs:
            gas_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                    loc[fuel])*dge_to_MJ*(CI_standard_linear.loc['Gasoline Compliance']\
                                                                          -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Gasoline'])

        for fuel in ldgas_subs:
            gas_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Vehicle Class','Detailed Fuel Type']).sum().\
                                                    loc['Light Duty'].loc[fuel])*dge_to_MJ*(CI_standard_linear.loc['Gasoline Compliance']\
                                                                                           -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Gasoline'])

        dsl_sub_table = pd.DataFrame(columns = listofyears)
        for fuel in diesel_subs:
            dsl_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                    loc[fuel])*dge_to_MJ*(CI_standard_linear.loc['Diesel Compliance']\
                                                                          -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Diesel'])

        for fuel in mdhd_diesel_subs:
            dsl_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Vehicle Class','Detailed Fuel Type']).sum().\
                                                    loc[['Medium Duty','Heavy Duty','Bus']].groupby('Detailed Fuel Type').sum()\
                                                    .loc[fuel])*dge_to_MJ*(CI_standard_linear.loc['Diesel Compliance']\
                                                                                           -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Diesel'])

        jet_sub_table = pd.DataFrame(columns = listofyears)

        jet_sub_table.loc['Renewable Jet Fuel'] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                loc['Renewable Jet Fuel'])*dge_to_MJ*(CI_standard_backloaded.loc['Jet Fuel Compliance']\
                                                                      -scenarioer['RNJ'].mean())



        credit_bank = pd.DataFrame(columns = listofyears)
        credit_bank.loc['Bank, Annual'] = gas_sub_table.sum() + dsl_sub_table.sum() + jet_sub_table.sum()
        credit_bank.loc['Bank, Cumulative'] =  credit_bank.loc['Bank, Annual'].cumsum()
        credit_bank /= 1000000


        fig = plt.figure()
        figtitle = scenarioname + ' Scenario Credit Bank' + "\n" + 'Linear ' + str(initial_target) + "%"
        savetitle = scenarioletter + ' Credit Bank, linear'
        ax = fig.gca(title = figtitle)
        ax.bar(credit_bank.T.index, credit_bank.loc['Bank, Annual'],color=(1.0, 0.0, 0.1, 1.0))
        ax.plot(credit_bank.T.index, credit_bank.loc['Bank, Cumulative'])
        ax.set_xlabel('Year')
        ax.set_ylabel('Credits')
        ax.grid()
        ax.set_axisbelow(True)
        ax.set_xticks(listofyears)
        ax.tick_params(axis = 'x', rotation = 90)
        ax.set_xlim(2019,2035)
        ax.legend(['Bank Cumulative', 'Bank, Annual'])
        fig.savefig(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".jpg",bbox_inches='tight')
        credit_bank.to_excel(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".xlsx")


        credits_deficits = pd.concat([gas_sub_table,dsl_sub_table,jet_sub_table])
        credits_deficits.index.name = "Fuel"
        credits_deficits = credits_deficits.groupby('Fuel').sum()

        toprint = credits_deficits[credits_deficits >5]/1000000
        toprint = toprint.fillna(0)
        test = toprint.sum(axis = 1)
        for fuel in test.index:
            if test.loc[fuel] <= 1:
                toprint = toprint.drop(fuel)

        infraorder = ['Gasoline', 'Ethanol','Diesel','Renewable Diesel','Biodiesel','Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen', 'Electricity','Renewable Jet Fuel']
        infracolor = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink','tab:gray','tab:olive','tab:cyan','darkgreen']
        colormap = zip(infraorder,infracolor)
        colormap = dict(colormap)
        indexorder = [x for x in infraorder if x in toprint.index]
        colors = [colormap[x] for x in colormap if x in indexorder]

        figtitle = scenarioname + ' Scenario Credits Generated' + "\n" + "Linear, " + str(initial_target) + "%"
        savetitle = scenarioletter + ' Credits Generated (linear)'
        ax = toprint.reindex(indexorder).T.plot(kind = 'bar',stacked = True, title = figtitle, color = colors)
        ax.set_xlabel('Year')
        ax.set_ylabel('Credits')
        ax.legend(title = False, bbox_to_anchor=(1,0.8), loc = 'upper left')
        ax.grid()
        ax.set_axisbelow(True)
        fig= ax.get_figure()
        fig.savefig(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".jpg",bbox_inches='tight')
        credits_deficits.reindex(infraorder).to_excel(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".xlsx")
        plt.close('all')

In [38]:
overallstart = time.time()
cwd = os.getcwd()
CI_compliance_read = pd.read_excel(cwd + "\\Inputs\\CI_Compliance_Input.xlsx")
CI_compliance_read.set_index('CI_Compliance',inplace = True)
EER_read = pd.read_excel(cwd + "\\Inputs\\EER.xlsx")
EER_read.set_index('EER', inplace = True)
file = cwd +"\\CO UserInterface_LIVE.xlsm"
graphfolder = cwd + "\\Results"
listofyears = list(range(2019,2036))
calyr = range(2019,2036)
modyr = [i for i in range(1961,2036)]
calyr = [i for i in range(2019,2036)]
methane_GWP = 28
n20_GWP = 265
dge_to_MJ = 134.47 #MJ/DGE
gge_to_MJ = 115.83 #MJ/GGE
gge_to_dge = 0.88 #DGE/GGE
kWh_to_MJ = 3.6 #MJ/kWh
pivotdictionary = {'ldv': 'Light Duty', 'mdv': 'Medium Duty', 'hdv':'Heavy Duty', 'GAS':'Gasoline',\
                   'DSL':'Diesel','ELC':'Electricity', 'NAG':'Fossil Natural Gas', 'HYD':'Hydrogen',\
                   'PHV': 'Plug-in Hybrid', 'pop':'sales', 'con':'Fuel Consumption', 'mpg':'Fuel Economy', 'vmt': 
                   'VMT', 'bus': 'Bus','JET': 'Jet Fuel', 'RNJ': 'Renewable Jet Fuel','ETH':'Ethanol',\
                   'RNG':'Renewable Natural Gas','RHD':'Renewable Hydrogen','RND':'Renewable Diesel','BID':'Biodiesel'}

new_dic = dict([[v,k] for k,v in pivotdictionary.items()])

wb = xw.Book(file)
eVMT = wb.sheets['Baseline Detailed Inputs'].range('B135').options(pd.DataFrame, expand = 'table').value
xw.apps.active.calculate()

scenariotitle = {'Baseline':'Baseline','A':'Low','B':'Medium','C':'High'}
scenariotargets = {'Baseline': 10, 'A':10, 'B':15,'C':20}


run = input("Scenario to run: ")

start = time.time()

    
if run == 'All':
    for scenarioletter in scenariotitle.keys():
        print("Running Scenario " + scenarioletter + "...")
        scenarioanalysis(scenarioletter)
        CI_credits(scenarioletter)
        initial_target = scenariotargets[scenarioletter]
        scenarioname = scenariotitle[scenarioletter]
#         plt.show()
# #     comparisongraphs()
# elif scenarioletter == 'Comparison':
#     comparisongraphs()

elif run == '':
    print()
    
else:
    scenarioletter = run
    print("Running Scenario " + scenarioletter + "...")
    initial_target = scenariotargets[scenarioletter]
    scenarioname = scenariotitle[scenarioletter]
    scenarioanalysis(scenarioletter)
    CI_credits(scenarioletter)

    
#     comparisongraphs()
    
plt.show()
print("Complete. Runtime: " + str(time.time()-start))

Scenario to run: 

Complete. Runtime: 0.0009963512420654297


In [39]:
print("Option to select scenario and adjust compliance target. Warning: figures will save as they are changed")

Option to select scenario and adjust compliance target. Warning: figures will save as they are changed


In [40]:
# CI calculation
plt.show()
scenariotargets = {'Baseline': 10, 'A':10, 'B':15,'C':20}

@interact
def choose_scenario(Scenario = ['A','B','C']):
    initial_target = scenariotargets[Scenario]
    @interact
    def choose_target(CI_2030_target= widgets.IntSlider(min=0, max=30, step=1, value=initial_target, description='Reduction Target:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d')):
        # CI_2030_target = 20
    
        scenarioletter = Scenario
        scenarioname = scenariotitle[scenarioletter]
        initial_target = scenariotargets[scenarioletter]
        CI_compliance = copy.deepcopy(CI_compliance_read)
        EER = copy.deepcopy(EER_read)
        CI_compliance *= CI_2030_target/100

        file = cwd +"\\PKL Files\\"+ scenarioletter + "ER.pkl"
        pkl_file = open(file,'rb')
        mydict = pickle.load(pkl_file)
        pkl_file.close()
        scenarioer = copy.deepcopy(mydict)

        file = cwd +"\\PKL Files\\"+ scenarioletter + "results.pkl"
        pkl_file = open(file,'rb')
        mydict = pickle.load(pkl_file)
        pkl_file.close()
        scenarioresults = copy.deepcopy(mydict)

        fuelpivottable = copy.deepcopy(scenarioresults['Fuel Consumption'])

        CI_standard_linear = pd.DataFrame(columns = list(range(2019,2036)))
        CI_standard_linear.loc['Gasoline Compliance'] = (1-CI_compliance.loc['Linear'])*scenarioer['GAS'].loc[2019,2019]
        CI_standard_linear.loc['Diesel Compliance'] = (1-CI_compliance.loc['Linear'])*scenarioer['DSL'].loc[2019,2019]
        CI_standard_linear.loc['Jet Fuel Compliance'] = (1-CI_compliance.loc['Linear'])*scenarioer['JET'].loc[2019,2019]
        CI_standard_linear.index.name = ['Linear Compliance [g/MJ]']
        CI_standard_linear.to_excel(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\Linear_Standard.xlsx")

        CI_standard_backloaded = pd.DataFrame(columns = list(range(2019,2036)))
        CI_standard_backloaded.loc['Gasoline Compliance'] = (1-CI_compliance.loc['Back-Loaded'])*scenarioer['GAS'].loc[2019,2019]
        CI_standard_backloaded.loc['Diesel Compliance'] = (1-CI_compliance.loc['Back-Loaded'])*scenarioer['DSL'].loc[2019,2019]
        CI_standard_backloaded.loc['Jet Fuel Compliance'] = (1-CI_compliance.loc['Back-Loaded'])*scenarioer['JET'].loc[2019,2019]
        CI_standard_backloaded.index.name = ['Back-Loaded Compliance [g/MJ]']
        CI_standard_backloaded.to_excel(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\BackLoaded_Standard.xlsx")

        #backloaded
        gas_subs = ['Gasoline','Ethanol']
        ldgas_subs = ['Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen','Electricity']

        diesel_subs = ['Diesel','Renewable Diesel','Biodiesel']
        mdhd_diesel_subs = ['Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen','Electricity']

        gas_sub_table = pd.DataFrame(columns = listofyears)
        for fuel in gas_subs:
            gas_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                    loc[fuel])*dge_to_MJ*(CI_standard_backloaded.loc['Gasoline Compliance']\
                                                                          -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Gasoline'])

        for fuel in ldgas_subs:
            gas_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Vehicle Class','Detailed Fuel Type']).sum().\
                                                    loc['Light Duty'].loc[fuel])*dge_to_MJ*(CI_standard_backloaded.loc['Gasoline Compliance']\
                                                                                           -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Gasoline'])

        dsl_sub_table = pd.DataFrame(columns = listofyears)
        for fuel in diesel_subs:
            dsl_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                    loc[fuel])*dge_to_MJ*(CI_standard_backloaded.loc['Diesel Compliance']\
                                                                          -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Diesel'])

        for fuel in mdhd_diesel_subs:
            dsl_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Vehicle Class','Detailed Fuel Type']).sum().\
                                                    loc[['Medium Duty','Heavy Duty','Bus']].groupby('Detailed Fuel Type').sum()\
                                                    .loc[fuel])*dge_to_MJ*(CI_standard_backloaded.loc['Diesel Compliance']\
                                                                                           -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Diesel'])

        #backloaded credit_deficit
        gas_subs = ['Gasoline','Ethanol']
        ldgas_subs = ['Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen','Electricity']

        diesel_subs = ['Diesel','Renewable Diesel','Biodiesel']
        mdhd_diesel_subs = ['Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen','Electricity']

        gas_sub_table = pd.DataFrame(columns = listofyears)
        for fuel in gas_subs:
            gas_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                    loc[fuel])*dge_to_MJ*(CI_standard_backloaded.loc['Gasoline Compliance']\
                                                                          -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Gasoline'])

        for fuel in ldgas_subs:
            gas_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Vehicle Class','Detailed Fuel Type']).sum().\
                                                    loc['Light Duty'].loc[fuel])*dge_to_MJ*(CI_standard_backloaded.loc['Gasoline Compliance']\
                                                                                           -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Gasoline'])


        jet_sub_table = pd.DataFrame(columns = listofyears)

        jet_sub_table.loc['Renewable Jet Fuel'] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                loc['Renewable Jet Fuel'])*dge_to_MJ*(CI_standard_backloaded.loc['Jet Fuel Compliance']\
                                                                      -scenarioer['RNJ'].mean())



        credit_bank = pd.DataFrame(columns = listofyears)
        credit_bank.loc['Bank, Annual'] = gas_sub_table.sum() + dsl_sub_table.sum() + jet_sub_table.sum()
        credit_bank.loc['Bank, Cumulative'] =  credit_bank.loc['Bank, Annual'].cumsum()
        credit_bank /= 1000000

        fig = plt.figure()
        figtitle = scenarioname + ' Scenario Credit Bank' + '\n' + 'Backloaded ' + str(CI_2030_target) + "%"
        savetitle = scenarioletter + ' Credit Bank, backloaded'
        ax = fig.gca(title = figtitle)
        ax.bar(credit_bank.T.index, credit_bank.loc['Bank, Annual'],color=(1.0, 0.0, 0.1, 1.0))
        ax.plot(credit_bank.T.index, credit_bank.loc['Bank, Cumulative'])
        ax.set_xlabel('Year')
        ax.set_ylabel('Credits')
        ax.grid()
        ax.set_axisbelow(True)
        ax.set_xticks(listofyears)
        ax.tick_params(axis = 'x', rotation = 90)
        ax.set_xlim(2019,2035)
        ax.legend(['Bank Cumulative', 'Bank, Annual'])
        fig.savefig(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".jpg",bbox_inches='tight')
        credit_bank.to_excel(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".xlsx")

        #backloaded credits generated
        credits_deficits = pd.concat([gas_sub_table,dsl_sub_table,jet_sub_table])
        credits_deficits.index.name = "Fuel"
        credits_deficits = credits_deficits.groupby('Fuel').sum()

        toprint = credits_deficits[credits_deficits >5]/1000000
        toprint = toprint.fillna(0)
        test = toprint.sum(axis = 1)
        for fuel in test.index:
            if test.loc[fuel] <= 1:
                toprint = toprint.drop(fuel)

        infraorder = ['Gasoline', 'Ethanol','Diesel','Renewable Diesel','Biodiesel','Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen', 'Electricity','Renewable Jet Fuel']
        infracolor = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink','tab:gray','tab:olive','tab:cyan','darkgreen']
        colormap = zip(infraorder,infracolor)
        colormap = dict(colormap)
        indexorder = [x for x in infraorder if x in toprint.index]
        colors = [colormap[x] for x in colormap if x in indexorder]

        figtitle = scenarioname + ' Scenario Credits Generated' + '\n' + 'Backloaded, ' + str(CI_2030_target) + "%"
        savetitle = scenarioletter + ' Credits Generated (backloaded)'
        ax = toprint.reindex(indexorder).T.plot(kind = 'bar',stacked = True, title = figtitle, color = colors)
        ax.set_xlabel('Year')
        ax.set_ylabel('Credits')
        ax.legend(title = False, bbox_to_anchor=(1,0.8), loc = 'upper left')
        ax.grid()
        ax.set_axisbelow(True)
        fig= ax.get_figure()
        fig.savefig(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".jpg",bbox_inches='tight')
        credits_deficits.reindex(infraorder).to_excel(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".xlsx")


        #linear

        gas_subs = ['Gasoline','Ethanol']
        ldgas_subs = ['Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen','Electricity']

        diesel_subs = ['Diesel','Renewable Diesel','Biodiesel']
        mdhd_diesel_subs = ['Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen','Electricity']

        gas_sub_table = pd.DataFrame(columns = listofyears)
        for fuel in gas_subs:
            gas_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                    loc[fuel])*dge_to_MJ*(CI_standard_linear.loc['Gasoline Compliance']\
                                                                          -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Gasoline'])

        for fuel in ldgas_subs:
            gas_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Vehicle Class','Detailed Fuel Type']).sum().\
                                                    loc['Light Duty'].loc[fuel])*dge_to_MJ*(CI_standard_linear.loc['Gasoline Compliance']\
                                                                                           -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Gasoline'])

        dsl_sub_table = pd.DataFrame(columns = listofyears)
        for fuel in diesel_subs:
            dsl_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                    loc[fuel])*dge_to_MJ*(CI_standard_linear.loc['Diesel Compliance']\
                                                                          -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Diesel'])

        for fuel in mdhd_diesel_subs:
            dsl_sub_table.loc[fuel] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Vehicle Class','Detailed Fuel Type']).sum().\
                                                    loc[['Medium Duty','Heavy Duty','Bus']].groupby('Detailed Fuel Type').sum()\
                                                    .loc[fuel])*dge_to_MJ*(CI_standard_linear.loc['Diesel Compliance']\
                                                                                           -scenarioer[new_dic[fuel]].mean()/EER.loc[fuel,'Diesel'])

        jet_sub_table = pd.DataFrame(columns = listofyears)

        jet_sub_table.loc['Renewable Jet Fuel'] = copy.deepcopy(fuelpivottable[listofyears].groupby(['Detailed Fuel Type']).sum().\
                                                loc['Renewable Jet Fuel'])*dge_to_MJ*(CI_standard_backloaded.loc['Jet Fuel Compliance']\
                                                                      -scenarioer['RNJ'].mean())



        credit_bank = pd.DataFrame(columns = listofyears)
        credit_bank.loc['Bank, Annual'] = gas_sub_table.sum() + dsl_sub_table.sum() + jet_sub_table.sum()
        credit_bank.loc['Bank, Cumulative'] =  credit_bank.loc['Bank, Annual'].cumsum()
        credit_bank /= 1000000


        fig = plt.figure()
        figtitle = scenarioname + ' Scenario Credit Bank' + "\n" + 'Linear ' + str(CI_2030_target) + "%"
        savetitle = scenarioletter + ' Credit Bank, linear'
        ax = fig.gca(title = figtitle)
        ax.bar(credit_bank.T.index, credit_bank.loc['Bank, Annual'],color=(1.0, 0.0, 0.1, 1.0))
        ax.plot(credit_bank.T.index, credit_bank.loc['Bank, Cumulative'])
        ax.set_xlabel('Year')
        ax.set_ylabel('Credits')
        ax.grid()
        ax.set_axisbelow(True)
        ax.set_xticks(listofyears)
        ax.tick_params(axis = 'x', rotation = 90)
        ax.set_xlim(2019,2035)
        ax.legend(['Bank Cumulative', 'Bank, Annual'])
        fig.savefig(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".jpg",bbox_inches='tight')
        credit_bank.to_excel(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".xlsx")


        credits_deficits = pd.concat([gas_sub_table,dsl_sub_table,jet_sub_table])
        credits_deficits.index.name = "Fuel"
        credits_deficits = credits_deficits.groupby('Fuel').sum()

        toprint = credits_deficits[credits_deficits >5]/1000000
        toprint = toprint.fillna(0)
        test = toprint.sum(axis = 1)
        for fuel in test.index:
            if test.loc[fuel] <= 1:
                toprint = toprint.drop(fuel)

        infraorder = ['Gasoline', 'Ethanol','Diesel','Renewable Diesel','Biodiesel','Fossil Natural Gas','Renewable Natural Gas','Hydrogen','Renewable Hydrogen', 'Electricity','Renewable Jet Fuel']
        infracolor = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink','tab:gray','tab:olive','tab:cyan','darkgreen']
        colormap = zip(infraorder,infracolor)
        colormap = dict(colormap)
        indexorder = [x for x in infraorder if x in toprint.index]
        colors = [colormap[x] for x in colormap if x in indexorder]

        figtitle = scenarioname + ' Scenario Credits Generated' + "\n" + "Linear, " + str(CI_2030_target) + "%"
        savetitle = scenarioletter + ' Credits Generated (linear)'
        ax = toprint.reindex(indexorder).T.plot(kind = 'bar',stacked = True, title = figtitle, color = colors)
        ax.set_xlabel('Year')
        ax.set_ylabel('Credits')
        ax.legend(title = False, bbox_to_anchor=(1,0.8), loc = 'upper left')
        ax.grid()
        ax.set_axisbelow(True)
        fig= ax.get_figure()
        fig.savefig(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".jpg",bbox_inches='tight')
        credits_deficits.reindex(infraorder).to_excel(graphfolder +"\\" + scenarioname  +"\\Credits_Deficits\\" +  savetitle + ".xlsx")

interactive(children=(Dropdown(description='Scenario', options=('A', 'B', 'C'), value='A'), Output()), _dom_cl…

In [41]:
#     file = cwd +"\\PKL Files\\"+ scenarioletter + "results.pkl"
#     pkl_file = open(file,'rb')
#     mydict = pickle.load(pkl_file)
#     pkl_file.close()
#     scenarioresults = copy.deepcopy(mydict)

In [42]:
# scenarioresults['Population'].groupby(['Fuel Type','Vehicle Class']).sum().loc['Fossil Natural Gas']

In [43]:
print("Select from drop-down below to view all saved graphs.")

Select from drop-down below to view all saved graphs.


In [44]:
fdir = cwd + "/Results/"
included_extensions = ['jpg','jpeg', 'bmp', 'png', 'gif']
@interact
def choose_scenario(Scenario=['Baseline','A']):
    @interact
    def choose_result(Result=['Credits_Deficits','Fuel','Emissions','VMT','Population','Sales']):
        file_names = [fn for fn in os.listdir(fdir+ Scenario +"/" +Result)
                      if any(fn.endswith(ext) for ext in included_extensions)]
        @interact
        def show_images(Graph=file_names):
            included_extensions = ['jpg','jpeg', 'bmp', 'png', 'gif']
            display(Image(fdir + Scenario +"/" + Result +"/"+ Graph))

interactive(children=(Dropdown(description='Scenario', options=('Baseline', 'A'), value='Baseline'), Output())…

In [45]:
oresults = {}

scenariographtitle = dict(Baseline = 'Baseline', A = "Low", B = 'Medium',\
                 C = 'High')

scenariotitle = dict(Baseline = 'Baseline', A = "Low", B = 'Medium',\
                 C = 'High')

for scenario in ['Baseline','A','B','C']:
    file = cwd +"\\PKL Files\\"+ scenario + "results.pkl"
    pkl_file = open(file,'rb')
    mydict = pickle.load(pkl_file)
    pkl_file.close()
    oresults[scenario] = copy.deepcopy(mydict)



scenariolist = list(oresults.keys())
#annual emissions comparison
#Annual emissions
annemissionscomparison = pd.DataFrame(columns = range(2019,2036), index = list(scenariographtitle.values())).fillna(0)
for scenario in scenariolist:
    annemissionscomparison.loc[scenariographtitle[scenario]] = oresults[scenario]['CO2'].loc['On'].sum()

savetitle = 'Annual Emission Comparison'
figtitle = 'Annual Emission Comparison' + '\n' +  '(on-road only)'
ax = annemissionscomparison.T.plot(title = figtitle)
ax.grid()
ax.set_axisbelow(True)
ax.set_xlabel('Year')
ax.set_xticks(range(2019,2036))
#     ax.set_yticks(range(5,28,2))
ax.set_xlim(2019,2035)
# ax.axhline(y = CO2target, c = 'black', label = '2050 GHG Target')
ax.tick_params(axis = 'x', labelrotation = 90)
plt.setp(ax.lines[0], marker = 'x', markevery = 2)
ax.legend(title = False, bbox_to_anchor=(1, 0.8), loc='upper left')
ax.set_ylabel('CO' + r'$_2$' + 'e Emissions [MMT]')
fig = ax.get_figure()
fig.savefig(graphfolder +"\\Comparison\\"+  savetitle + ".jpg",bbox_inches='tight')
annemissionscomparison.to_excel(graphfolder +"\\Comparison\\"+  savetitle + ".xlsx")

baseline = copy.deepcopy(annemissionscomparison.loc['Baseline'])
annemissionspercent = (annemissionscomparison - baseline)/baseline
annemissionspercent *= 100
annemissionspercent = annemissionspercent[[2030,2035]]
annemissionspercent.index.name = 'Emissions Reduction from Baseline [%]'
annemissionspercent.to_excel(graphfolder +"\\Comparison\\Percent Change.xlsx")
annemissionspercent[[2030,2035]]

,2030,2035
Emissions Reduction from Baseline [%],,
Baseline,0.000000,0.000000
Low,-2.914938,-3.772368
Medium,-5.903514,-9.438277
High,-8.215386,-14.475279
